In [ ]:
import os
from dotenv import load_dotenv
import nest_asyncio
import asyncio
import aiohttp
import pandas as pd
import warnings
from google.oauth2 import service_account
import pandas_gbq
warnings.filterwarnings("ignore")
pd.set_option("display.max_columns", None)
load_dotenv()
nest_asyncio.apply()
BASE_URL = "https://api.themoviedb.org/3"
ENDPOINT_TRENDING_MOVIE_WEEK = "/trending/movie/week"
TMDB_IMAGE_BASE = "https://image.tmdb.org/t/p/"
POSTER_SIZE = "w185"

In [2]:
load_dotenv("C:\\Users\\ALIF\\Documents\\Kelas JCDSBDGPM-10\\PORTO\\.env")

True

In [3]:
API_KEY = os.getenv("TMDB_API_KEY")

In [ ]:
async def fetch_page(session: aiohttp.ClientSession, page: int) -> dict:
    url = BASE_URL + ENDPOINT_TRENDING_MOVIE_WEEK
    params = {"api_key": API_KEY, "page": page}

    async with session.get(url, params=params) as resp:
        status = resp.status
        data = await resp.json()

        print(f"Ambil page {page} ... status code: {status}")

        if status != 200:
            raise RuntimeError(f"Gagal ambil page {page}: status {status} | body: {data}")

        return data


async def fetch_trending_movie_week_async(
    max_pages: int | None = None,
    concurrency: int = 4
) -> pd.DataFrame:
    connector = aiohttp.TCPConnector(limit=None)
    async with aiohttp.ClientSession(connector=connector) as session:

        first_data = await fetch_page(session, page=1)
        total_pages_api = first_data["total_pages"]
        total_results = first_data["total_results"]

        if max_pages is None:
            n_pages = total_pages_api
        else:
            n_pages = min(max_pages, total_pages_api)

        print(f"\nTotal pages di API : {total_pages_api}")
        print(f"Total results di API : {total_results}")
        print(f"Pages yang di-fetch : 1..{n_pages}\n")

        all_results = list(first_data["results"])

        if n_pages == 1:
            df = pd.DataFrame(all_results)
            return df

        semaphore = asyncio.Semaphore(concurrency)

        async def bound_fetch(page: int) -> dict:
            async with semaphore:
                return await fetch_page(session, page)

        tasks = [
            asyncio.create_task(bound_fetch(page))
            for page in range(2, n_pages + 1)
        ]

        pages_data = await asyncio.gather(*tasks)

        for page_data in pages_data:
            all_results.extend(page_data["results"])

        df = pd.DataFrame(all_results)
        return df

In [ ]:
# Jalankan coroutine dan dapatkan semua data
df_trending_all = await fetch_trending_movie_week_async(
    max_pages=None,      # ambil semua sampai total_pages (biasanya 500)
    concurrency=4        # boleh kamu ubah kalau mau
)

print("Shape sebelum dedup:", df_trending_all.shape)
display(df_trending_all.head())

before = len(df_trending_all)
df_trending_all = (
    df_trending_all
    .drop_duplicates(subset="id", keep="first")
    .reset_index(drop=True)
)
after = len(df_trending_all)

print(f"Baris sebelum : {before}")
print(f"Baris sesudah : {after}")
print(f"Duplikat dibuang: {before - after}")
print("Masih ada duplikat id?", df_trending_all["id"].duplicated().any())

kolom_penting = [
    "id",
    "title",
    "original_title",
    "overview",
    "release_date",
    "vote_average",
    "vote_count",
    "popularity",
    "media_type",
    "original_language",
    "poster_path",     
]

kolom_penting_ada = [k for k in kolom_penting if k in df_trending_all.columns]

df_trending_clean = df_trending_all[kolom_penting_ada].copy()

def make_poster_url(path):
    if not path or pd.isna(path):
        return None
    return f"{TMDB_IMAGE_BASE}{POSTER_SIZE}{path}"

df_trending_clean["poster_url"] = df_trending_clean["poster_path"].apply(make_poster_url)

df_trending_clean.head()

Ambil page 1 ... status code: 200

Total pages di API : 500
Total results di API : 10000
Pages yang di-fetch : 1..500

Ambil page 2 ... status code: 200
Ambil page 6 ... status code: 200
Ambil page 3 ... status code: 200
Ambil page 4 ... status code: 200
Ambil page 5 ... status code: 200
Ambil page 7 ... status code: 200
Ambil page 8 ... status code: 200
Ambil page 9 ... status code: 200
Ambil page 10 ... status code: 200
Ambil page 11 ... status code: 200
Ambil page 12 ... status code: 200
Ambil page 13 ... status code: 200
Ambil page 14 ... status code: 200
Ambil page 15 ... status code: 200
Ambil page 16 ... status code: 200
Ambil page 17 ... status code: 200
Ambil page 18 ... status code: 200
Ambil page 19 ... status code: 200
Ambil page 21 ... status code: 200
Ambil page 22 ... status code: 200
Ambil page 20 ... status code: 200
Ambil page 23 ... status code: 200
Ambil page 24 ... status code: 200
Ambil page 25 ... status code: 200
Ambil page 26 ... status code: 200
Ambil page 27 

,adult,backdrop_path,id,title,original_title,overview,poster_path,media_type,original_language,genre_ids,popularity,release_date,video,vote_average,vote_count
0,False,/5h2EsPKNDdB3MAtOk9MB9Ycg9Rz.jpg,1084242,Zootopia 2,Zootopia 2,After cracking the biggest case in Zootopia's ...,/oJ7g2CifqpStmoYQyaLQgEU32qO.jpg,movie,en,"[16, 10751, 35, 12, 9648]",372.4120,2025-11-26,False,7.559,76
1,False,/uPvsuTYc2fgEYgp3Ib2INoGep0n.jpg,701387,Bugonia,Bugonia,Two conspiracy obsessed young men kidnap the h...,/oxgsAQDAAxA92mFGYCZllgWkH9J.jpg,movie,en,"[35, 80, 878]",109.3434,2025-10-23,False,7.610,451
2,False,/l8pwO23MCvqYumzozpxynCNfck1.jpg,967941,Wicked: For Good,Wicked: For Good,As an angry mob rises against the Wicked Witch...,/si9tolnefLSUKaqQEGz1bWArOaL.jpg,movie,en,"[14, 12, 10749]",230.2695,2025-11-19,False,6.790,274
3,False,/7nAVXGHHtaNcdsqvDXmY6R9N0fG.jpg,1363123,The Family Plan 2,The Family Plan 2,"Now that Dan's assassin days are behind him, a...",/semFxuYx6HcrkZzslgAkBqfJvZk.jpg,movie,en,"[28, 35]",374.8090,2025-11-11,False,6.821,196
4,False,/u5NWHVhZ6HWc1fXnqn82cMar5St.jpg,1241983,Train Dreams,Train Dreams,A logger leads a life of quiet grace as he exp...,/l3zS4YnpOi4usyEXGJMtxSqDDyb.jpg,movie,en,[18],97.5006,2025-11-05,False,7.388,139


Baris sebelum : 9985
Baris sesudah : 9785
Duplikat dibuang: 200
Masih ada duplikat id? False


,id,title,original_title,overview,release_date,vote_average,vote_count,popularity,media_type,original_language,poster_path,poster_url
0,1084242,Zootopia 2,Zootopia 2,After cracking the biggest case in Zootopia's ...,2025-11-26,7.559,76,372.4120,movie,en,/oJ7g2CifqpStmoYQyaLQgEU32qO.jpg,https://image.tmdb.org/t/p/w185/oJ7g2CifqpStmo...
1,701387,Bugonia,Bugonia,Two conspiracy obsessed young men kidnap the h...,2025-10-23,7.610,451,109.3434,movie,en,/oxgsAQDAAxA92mFGYCZllgWkH9J.jpg,https://image.tmdb.org/t/p/w185/oxgsAQDAAxA92m...
2,967941,Wicked: For Good,Wicked: For Good,As an angry mob rises against the Wicked Witch...,2025-11-19,6.790,274,230.2695,movie,en,/si9tolnefLSUKaqQEGz1bWArOaL.jpg,https://image.tmdb.org/t/p/w185/si9tolnefLSUKa...
3,1363123,The Family Plan 2,The Family Plan 2,"Now that Dan's assassin days are behind him, a...",2025-11-11,6.821,196,374.8090,movie,en,/semFxuYx6HcrkZzslgAkBqfJvZk.jpg,https://image.tmdb.org/t/p/w185/semFxuYx6HcrkZ...
4,1241983,Train Dreams,Train Dreams,A logger leads a life of quiet grace as he exp...,2025-11-05,7.388,139,97.5006,movie,en,/l3zS4YnpOi4usyEXGJMtxSqDDyb.jpg,https://image.tmdb.org/t/p/w185/l3zS4YnpOi4usy...


In [ ]:
async def get_genre_mapping_async(language: str = "en-US") -> dict:
    """
    Ambil daftar genre movie dari TMDB.
    
    Return:
        dict {genre_id: genre_name}
    """
    url = BASE_URL + "/genre/movie/list"
    params = {"api_key": API_KEY, "language": language}

    async with aiohttp.ClientSession() as session:
        async with session.get(url, params=params) as resp:
            status = resp.status
            data = await resp.json()
            print(f"Ambil genre list ... status code: {status}")

            if status != 200:
                raise RuntimeError(
                    f"Gagal ambil genre list: status {status} | body: {data}"
                )

    genres = data.get("genres", [])
    return {g["id"]: g["name"] for g in genres}

genre_map = await get_genre_mapping_async(language="en-US")

genre_map

Ambil genre list ... status code: 200


{28: 'Action',
 12: 'Adventure',
 16: 'Animation',
 35: 'Comedy',
 80: 'Crime',
 99: 'Documentary',
 18: 'Drama',
 10751: 'Family',
 14: 'Fantasy',
 36: 'History',
 27: 'Horror',
 10402: 'Music',
 9648: 'Mystery',
 10749: 'Romance',
 878: 'Science Fiction',
 10770: 'TV Movie',
 53: 'Thriller',
 10752: 'War',
 37: 'Western'}

In [7]:
# --- Buat kolom 'genres' (nama genre berdasarkan 'genre_ids') ---

def map_genre_ids_to_names(genre_ids):
    """
    genre_ids: list of int, misalnya [28, 12]
    return: string, misalnya "Action, Adventure"
    """
    if not isinstance(genre_ids, (list, tuple)):
        return None
    names = [genre_map.get(gid) for gid in genre_ids if gid in genre_map]
    if not names:
        return None
    return ", ".join(names)

df_trending_all["genres"] = df_trending_all["genre_ids"].apply(map_genre_ids_to_names)

df_trending_all[["title", "genre_ids", "genres"]].head()

,title,genre_ids,genres
0,Zootopia 2,"[16, 10751, 35, 12, 9648]","Animation, Family, Comedy, Adventure, Mystery"
1,Bugonia,"[35, 80, 878]","Comedy, Crime, Science Fiction"
2,Wicked: For Good,"[14, 12, 10749]","Fantasy, Adventure, Romance"
3,The Family Plan 2,"[28, 35]","Action, Comedy"
4,Train Dreams,[18],Drama


In [8]:
# Movie Indonesia
df_movie_indo = df_trending_clean[
    df_trending_clean["original_language"] == "id"
].copy()
df_movie_indo["segment"] = "indonesia"

# Movie Global
df_movie_global = df_trending_clean[
    df_trending_clean["original_language"] != "id"
].copy()
df_movie_global["segment"] = "global"

In [9]:
kolom_penting = [
    "id",
    "title",              # nama movie
    "overview",
    "release_date",
    "vote_average",
    "vote_count",
    "popularity",
    "media_type",         # di trending movie/week isinya "movie"
    "original_language",
    "poster_path",
    "genre_ids",
    "genres",             # kolom nama genre hasil mapping
]

kolom_penting_ada = [k for k in kolom_penting if k in df_trending_all.columns]
df_trending_clean = df_trending_all[kolom_penting_ada].copy()

df_trending_clean.head()

,id,title,overview,release_date,vote_average,vote_count,popularity,media_type,original_language,poster_path,genre_ids,genres
0,1084242,Zootopia 2,After cracking the biggest case in Zootopia's ...,2025-11-26,7.559,76,372.4120,movie,en,/oJ7g2CifqpStmoYQyaLQgEU32qO.jpg,"[16, 10751, 35, 12, 9648]","Animation, Family, Comedy, Adventure, Mystery"
1,701387,Bugonia,Two conspiracy obsessed young men kidnap the h...,2025-10-23,7.610,451,109.3434,movie,en,/oxgsAQDAAxA92mFGYCZllgWkH9J.jpg,"[35, 80, 878]","Comedy, Crime, Science Fiction"
2,967941,Wicked: For Good,As an angry mob rises against the Wicked Witch...,2025-11-19,6.790,274,230.2695,movie,en,/si9tolnefLSUKaqQEGz1bWArOaL.jpg,"[14, 12, 10749]","Fantasy, Adventure, Romance"
3,1363123,The Family Plan 2,"Now that Dan's assassin days are behind him, a...",2025-11-11,6.821,196,374.8090,movie,en,/semFxuYx6HcrkZzslgAkBqfJvZk.jpg,"[28, 35]","Action, Comedy"
4,1241983,Train Dreams,A logger leads a life of quiet grace as he exp...,2025-11-05,7.388,139,97.5006,movie,en,/l3zS4YnpOi4usyEXGJMtxSqDDyb.jpg,[18],Drama


In [10]:
# Movie Indonesia
df_movie_id = df_trending_clean[
    df_trending_clean["original_language"] == "id"
].copy()
df_movie_id["segment"] = "indonesia"

# Movie Global (selain Indonesia)
df_movie_global = df_trending_clean[
    df_trending_clean["original_language"] != "id"
].copy()
df_movie_global["segment"] = "global"

print("Jumlah movie Indonesia :", len(df_movie_id))
print("Jumlah movie global    :", len(df_movie_global))

display(df_movie_id.head())
display(df_movie_global.head())

Jumlah movie Indonesia : 53
Jumlah movie global    : 9732


,id,title,overview,release_date,vote_average,vote_count,popularity,media_type,original_language,poster_path,genre_ids,genres,segment
561,1391153,The Book of Sijjin & Illiyyin,"As a child, Yuli witnessed her parents die at ...",2025-07-17,7.4,4,12.3790,movie,id,/5c8iXGH5Vol8HfpdwXK5ptHKQ6R.jpg,"[27, 53]","Horror, Thriller",indonesia
715,1306525,The Elixir,An elixir unleashes the undead in a village. A...,2025-10-22,6.0,141,14.0832,movie,id,/A6aJLPhtmin9ZTWC2h7dnrMHU4z.jpg,"[27, 53]","Horror, Thriller",indonesia
753,1287571,Agak Laen: Menyala Pantiku!,"After repeatedly failing their missions, Detec...",2025-11-27,8.5,2,17.5012,movie,id,/48Lbiwc8QiaGXhpfHkWExLgIcaa.jpg,"[35, 80, 18, 9648]","Comedy, Crime, Drama, Mystery",indonesia
1026,1393334,Smothered,"After surviving an accident, Alif, a micro-pai...",2025-11-27,7.0,1,2.5688,movie,id,/5qX69jou3zFKjdyp4b4tAK0jGfD.jpg,"[18, 9648, 53]","Drama, Mystery, Thriller",indonesia
1122,1385536,Sore: A Wife From The Future,A young man living alone in Croatia encounters...,2025-07-10,8.9,15,5.2051,movie,id,/u4pNXPmBuYeTtksakUCZgJ1zpSB.jpg,"[14, 10749, 18, 878]","Fantasy, Romance, Drama, Science Fiction",indonesia


,id,title,overview,release_date,vote_average,vote_count,popularity,media_type,original_language,poster_path,genre_ids,genres,segment
0,1084242,Zootopia 2,After cracking the biggest case in Zootopia's ...,2025-11-26,7.559,76,372.4120,movie,en,/oJ7g2CifqpStmoYQyaLQgEU32qO.jpg,"[16, 10751, 35, 12, 9648]","Animation, Family, Comedy, Adventure, Mystery",global
1,701387,Bugonia,Two conspiracy obsessed young men kidnap the h...,2025-10-23,7.610,451,109.3434,movie,en,/oxgsAQDAAxA92mFGYCZllgWkH9J.jpg,"[35, 80, 878]","Comedy, Crime, Science Fiction",global
2,967941,Wicked: For Good,As an angry mob rises against the Wicked Witch...,2025-11-19,6.790,274,230.2695,movie,en,/si9tolnefLSUKaqQEGz1bWArOaL.jpg,"[14, 12, 10749]","Fantasy, Adventure, Romance",global
3,1363123,The Family Plan 2,"Now that Dan's assassin days are behind him, a...",2025-11-11,6.821,196,374.8090,movie,en,/semFxuYx6HcrkZzslgAkBqfJvZk.jpg,"[28, 35]","Action, Comedy",global
4,1241983,Train Dreams,A logger leads a life of quiet grace as he exp...,2025-11-05,7.388,139,97.5006,movie,en,/l3zS4YnpOi4usyEXGJMtxSqDDyb.jpg,[18],Drama,global


In [11]:
project_id = "the-movies-weekly"
dataset_id = "movie_analytics"

# path file service account, sesuaikan dengan punyamu
pathjsonnn = "C:\\Users\\ALIF\\Documents\\Kelas JCDSBDGPM-10\\PORTO\\the-movies-weekly-bc77a4292aef.json"
SERVICE_ACCOUNT_FILE = pathjsonnn

credentials = service_account.Credentials.from_service_account_file(
    SERVICE_ACCOUNT_FILE
)

pandas_gbq.context.credentials = credentials
pandas_gbq.context.project = project_id

In [ ]:
table_indo = f"{dataset_id}.trending_movie_week_indo"

pandas_gbq.to_gbq(
    df_movie_id,
    destination_table=table_indo,
    project_id=project_id,
    if_exists="replace"
)

table_global = f"{dataset_id}.trending_movie_week_global"

pandas_gbq.to_gbq(
    df_movie_global,
    destination_table=table_global,
    project_id=project_id,
    if_exists="replace"
)

100%|██████████| 1/1 [00:00<00:00, 12446.01it/s]
